# Construyamos un App con Gen AI ✨

En este taller vamos a aprender a utilizar grandes modelos de lenguaje de manera local, y a construir una pequeña aplicación encima de ese servicio.

In [ ]:
## 0. Instalar Ollama